In [1]:
import os
from collections import defaultdict, Counter

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from graphviz import Digraph
from scipy.spatial import distance
from sklearn.cluster import MiniBatchKMeans
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import   Dense

from hse_dialog_tree.kmeans_names import get_names
from hse_dialog_tree.utils.cpu import get_processor_info
from hse_dialog_tree.utils.files import load_pickle, dump_pickle
from hse_dialog_tree.text_vectorizers import TfUniversalSentenceEncoder

LANG = 'rus'
print(get_processor_info())

Intel64 Family 6 Model 142 Stepping 9, GenuineIntel x 4


In [2]:
sentence_encoder = TfUniversalSentenceEncoder('Z:/Общие диски/Диплом Дерево диалогов/models/universal-sentence-encoder-multilingual-large_3')

In [3]:
dramas = load_pickle(f'Z:/Общие диски/Диплом Дерево диалогов/data/{LANG}/content_all.pkl.zip')

dramas_texts = []
for k, v in dramas.items():
    dramas_texts.append(v)

In [4]:
vectors = load_pickle(f'Z:/Общие диски/Диплом Дерево диалогов/data/{LANG}/02_vectors_v2.pkl.zip')
sentence_to_vector = load_pickle(f'Z:/Общие диски/Диплом Дерево диалогов/data/{LANG}/02_sentence_to_vector_v2.pkl.zip')

cluster_model_kmeans = load_pickle('Z:/Общие диски/Диплом Дерево диалогов/steps/03_kmeans/kmeans_v2_044.pkl.zip')
cluster_names = get_names(cluster_model_kmeans)
cluster_names = {
        0: 'Благодарность, радость, приветствие',
        1: 'Короткая нейтральная фраза',
        2: 'Ответ',
        3: 'Односложное восклицание',
        4: 'Мысли, рассуждение',
        5: 'Риторический вопрос, вопрос, порицание',
        6: 'Объяснение, пояснение',
        7: 'Негативное восклицание',
        8: 'Место, направление',
        9: 'Просьба, мольба',
        10: 'Обращение, мудрые изречения',
        11: 'Просьба уйти',
        12: 'Вопрос, предложение с "как"',
        13: 'Краткий ответ',
        14: 'Воля, честь, отвага',
        15: 'Небольшое повествование',
        16: 'Суждение',
        17: '"Знаю"',
        18: '"Хорошо", "молодец"',
        19: 'Короткий вопрос',
        20: 'Повествование, высокий слог',
        21: 'Думать, вспомнить',
        22: 'Личная фраза (я, ты, мы)',
        23: 'Смерть',
        24: 'Царь',
        25: 'Хозяин, барин, сударь',
        26: 'Смотреть, видеть',
        27: 'Короткий вопрос (кто, что, как)',
        28: 'Восклицание с порицанием',
        29: 'Отрицание',
        30: 'Обращения',
        31: 'Отец, сын',
        32: 'Испуг, страх',
        33: 'Нейтральная фраза',
        34: 'О семье (брат/сестра/дядя/дед)',
        35: 'Короткая фраза с многоточием',
        36: 'О любви',
        37: 'Короткий вопрос, недопонимание',
        38: 'Короткий вопрос обыденный',
        39: 'Удивление',
        40: 'Длинное рассуждение',
        41: 'Жених, невеста',
        42: 'О времени',
        43: 'Друзья',
    }

In [5]:
def ngrams_from_vector(vec, n=3):
    return [vec[i: i+n] for i in range(len(vec)-n + 1)]

def ngrams_from_vector2(vec, text, n=3):
    return [(vec[i: i+n], text[i: i+n]) for i in range(len(vec)-n + 1)]

In [6]:
cluster_vecs = []
cluster_vecs_text_no = []
cluster_vecs_text = []
for i, drama in enumerate(tqdm(dramas_texts)):
    for part in drama:
        clusters = []
        texts = []
        for person, sent in part:
            if sent not in sentence_to_vector:
                continue # Skip empty or too long texts
            sent_vec = sentence_to_vector[sent]
            clusters.append(sent_vec)
            texts.append(sent)
        cluster_vecs.append(clusters)
        cluster_vecs_text_no.append(i)
        cluster_vecs_text.append(texts)

100%|███████████████████████████████████████████████████████████████████████████████| 210/210 [00:00<00:00, 785.46it/s]


In [7]:
cluster_vecs_text_no_train, _ = train_test_split(list(set(cluster_vecs_text_no)), test_size=0.2, random_state=42)

In [8]:
len(cluster_vecs), len(cluster_vecs_text_no), len(cluster_vecs_text)

(5869, 5869, 5869)

In [9]:
X_lstm_train, X_lstm_test, y_lstm_train, y_lstm_test = [], [], [],[]
X_text_train = []
cl_lstm_train, cl_lstm_test = [], []

for vec, text_no, text in zip(tqdm(cluster_vecs), cluster_vecs_text_no, cluster_vecs_text):
    if len(vec) < 3:
        continue
    for ngrams, texxts in ngrams_from_vector2(vec, text, n=8):
        X_lstm = ngrams[:3].copy()
        y_lstm = ngrams[3].copy()
        cl_lstm = cluster_model_kmeans.predict(np.array(ngrams[3:]))
        if text_no in cluster_vecs_text_no_train:
            X_lstm_train.append(X_lstm)
            y_lstm_train.append(y_lstm)
            cl_lstm_train.append(cl_lstm)
            X_text_train.append(texxts)
        else:
            X_lstm_test.append(X_lstm)
            y_lstm_test.append(y_lstm)
            cl_lstm_test.append(cl_lstm)

X_lstm_train = np.array(X_lstm_train)
X_lstm_test = np.array(X_lstm_test)
y_lstm_train = np.array(y_lstm_train)
y_lstm_test = np.array(y_lstm_test)
cl_lstm_train = np.array(cl_lstm_train)
cl_lstm_test = np.array(cl_lstm_test)
print(len(X_lstm_train))     

100%|██████████████████████████████████████████████████████████████████████████████| 5869/5869 [01:25<00:00, 68.41it/s]


131153


In [10]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
import tensorflow as tf

In [11]:
dump_pickle(
    (X_lstm_train, y_lstm_train, X_lstm_test, y_lstm_test, cl_lstm_train, cl_lstm_test),
    '03_train_recurrent3.pkl.zip'
)